# Basic GAN
---

Check that we don't have python 3.10.2 (otherwise PyTorch won't work)

In [1]:
import sys
print(sys.version)

3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]


---
Imports

In [2]:
# Imports
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

True


In [3]:
import time

start_time = time.time()

---
Hyperparameters
- bs = batch size
- lr = learning rate

In [4]:
bs = 100
learning_rate_g = 0.0002 
learning_rate_d = 0.0002
threshold_no_train = 0.1
dropout_generator = 0.3
dropout_discriminator = 0.3
double_dropout_generator = False

---
Loading the dataset

In [5]:
# MNIST Dataset
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
# the output of torchvision datasets are PILImage images of range [0, 1] and we 
# want data that is centered around 0 with a std of 1 (0.1307 and 0.3081 are the estimated values of the MNIST mean & std)

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
print(f"Size train_dataset: {len(train_dataset)}")
min_value_dataset = float('inf')
max_value_dataset = -float('inf')
for image_idx in range(len(train_dataset)):
    min_value_dataset = min(train_dataset[image_idx][0].min(), min_value_dataset)
    max_value_dataset = max(train_dataset[image_idx][0].max(), max_value_dataset)
print(min_value_dataset)
print(max_value_dataset)

test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=True)
print(f"Size train_dataset: {len(test_dataset)}")

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

Size train_dataset: 60000
tensor(-0.4242)
tensor(2.8215)
Size train_dataset: 10000



---

Generator & discriminator

In [6]:
# Generator
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # Forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, dropout_generator)
        x = F.leaky_relu(self.fc2(x), 0.2)
        if double_dropout_generator:
            x = F.dropout(x, dropout_generator)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.tanh(self.fc4(x))  # dans [-1; 1]
        x = ((min_value_dataset + max_value_dataset) / 2) + ((max_value_dataset - min_value_dataset) / 2) * x
        return x  # dans [-min_value_dataset; max_value_dataset]

    
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    # Forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, dropout_discriminator)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, dropout_discriminator)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, dropout_discriminator)
        return torch.sigmoid(self.fc4(x)) # output dans [0; 1]

In [7]:
# Build network
z_dim = 100  # latent space dimension, input of the generator
mnist_dim = train_dataset.data.size(1) * train_dataset.data.size(2)
# drawings dimension, output of the generator and input of the discriminator
# cf torch_empty_and_tensor_dot_size.py for correct use of the .size method

print(f"latent space dimension: {z_dim}")
print(f"drawings dimension: {mnist_dim}")
print(f"28*28 = {28*28}")

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(d_input_dim =  mnist_dim).to(device)

latent space dimension: 100
drawings dimension: 784
28*28 = 784


In [8]:
G

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [9]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [10]:
# Loss
criterion = nn.BCELoss()

# Optimizer
G_optimizer = optim.Adam(G.parameters(), lr = learning_rate_g)
D_optimizer = optim.Adam(D.parameters(), lr = learning_rate_d)

In [11]:
def D_train(x):
    """
    Train the discriminator on 2*bs samples, 1*bs real and 1*bs fake data
    :param: x: real data
    :return: discriminator loss for the batch, has the discriminator been trained this batch, x_fake (to
             train the generator - it'd be computationally heavy to re-generate latent noise and re-applying G(z))
    """
    # train discriminator on real data
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    
    # train discriminator on fake data
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)

    # gradient backprop & optimize ONLY D's parameters
    D_loss = (D_real_loss + D_fake_loss) / 2
    
    to_train = D_loss > threshold_no_train
    
    if to_train:
        D_loss.backward(retain_graph = True)  # retain_graph because we are going to use x_fake again
        D_optimizer.step()
        
    return D_loss.data.item(), to_train, x_fake # the item() method extracts the loss’s value as a Python float

In [12]:
def G_train(x_fake):
    """
    Train the generator on bs sample
    :param: x_fake: G(z) already calculated in D_train
    """
    y = Variable(torch.ones(bs, 1).to(device))
    
    D_output = D(x_fake)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()  # the item() method extracts the loss’s value as a Python float

---
Time to initialise everyting

In [13]:
end_initialise = time.time()
print("Initialisation over:", round(end_initialise - start_time, 2), "s")

Initialisation over: 16.25 s


In [ ]:
n_epoch = 200
for epoch in range(1, n_epoch+1):
    if epoch % 5 == 1:
        with torch.no_grad():
            G.eval()
            test_z = Variable(torch.randn(bs, z_dim).to(device))
            generated = G(test_z)

            save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + str(epoch-1) + '.png')
    
    G.train()
    D_losses, D_was_trained_history, G_losses = [], [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        D.zero_grad()
        G.zero_grad()
        
        D_loss, D_was_trained, x_fake = D_train(x)
        D_losses.append(D_loss)
        D_was_trained_history.append(D_was_trained)
        
        G_losses.append(G_train(x_fake))
    
    
    D_was_trained_count = 0
    for booleen in D_was_trained_history:
        if booleen:
            D_was_trained_count += 1
    loss_g = torch.mean(torch.FloatTensor(G_losses))
    loss_d = torch.mean(torch.FloatTensor(D_losses))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f, discriminator trained: %d/%d, average epoch time: %.2f s' % (
            epoch, n_epoch, loss_d, loss_g, 
            D_was_trained_count, math.ceil(len(train_dataset)/bs), (time.time()-end_initialise)/epoch))

[1/200]: loss_d: 0.160, loss_g: 6.027, discriminator trained: 174/600, average epoch time: 28.20 s
[2/200]: loss_d: 0.081, loss_g: 4.669, discriminator trained: 10/600, average epoch time: 27.11 s
[3/200]: loss_d: 0.073, loss_g: 3.750, discriminator trained: 6/600, average epoch time: 26.66 s
[4/200]: loss_d: 0.112, loss_g: 3.880, discriminator trained: 100/600, average epoch time: 26.67 s
[5/200]: loss_d: 0.081, loss_g: 4.839, discriminator trained: 141/600, average epoch time: 26.82 s
[6/200]: loss_d: 0.093, loss_g: 9.105, discriminator trained: 228/600, average epoch time: 27.09 s
[7/200]: loss_d: 0.077, loss_g: 6.943, discriminator trained: 104/600, average epoch time: 27.16 s
[8/200]: loss_d: 0.062, loss_g: 5.721, discriminator trained: 70/600, average epoch time: 27.09 s
[9/200]: loss_d: 0.046, loss_g: 6.136, discriminator trained: 9/600, average epoch time: 27.01 s
[10/200]: loss_d: 0.038, loss_g: 5.741, discriminator trained: 0/600, average epoch time: 26.87 s
[11/200]: loss_d:

---
Save models

In [ ]:
torch.save(G.state_dict(), 'G.pt')
torch.save(D.state_dict(), 'D.pt')

---
Total time to run all the epochs

In [ ]:
print("Total time:", round(time.time() - start_time, 2), "s")

---
Load the model

In [ ]:
G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
G.load_state_dict(torch.load('G.pt'))
G.eval()

test_z = Variable(torch.randn(bs, z_dim).to(device))
generated = G(test_z)
save_image(generated.view(generated.size(0), 1, 28, 28), './samples/final.png')